In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
import logging
import numpy as np
import socket
hostname = socket.gethostname()

import sys
import mxnet as mx

In [13]:
stack = 'MD589'

In [ ]:
# for sec in range(154, 169):

for sec in [168]:

    print sec

    if hostname.startswith('gcn'):
        data_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/%(stack)s_byROI/%(sec)04d/roi1'%{'stack':stack,
                                                                                                        'sec': sec}
    elif hostname == 'yuncong-MacbookPro':
        data_dir = '/home/yuncong/CSHL_data_patches/%(stack)s_byROI/%(sec)04d/roi1'%{'stack':stack,'sec': sec}

    prefix = 'inception-stage1'
    n_iter = 6
    
    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
    
    img_filenames = sorted(os.listdir(data_dir))
    with open('/home/yuncong/CSHL_data_patches/%(dataset)s_test.lst' % \
              {'dataset': dataset}, 'w') as lst_file:
        for index, img_filename in enumerate(img_filenames):
            lst_file.write('%s\t0\t%s\n'%(index, img_filename))
            
    os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/%(dataset)s_test.lst \
%(data_dir)s/ /home/yuncong/CSHL_data_patches/%(dataset)s_test.rec' % {'dataset': dataset, 'data_dir': data_dir})
    
# on mbp, scanning 1000 images take 1.8s
# on gordon, scanning 16 image, 0.06s


In [10]:
if hostname.startswith('gcn'):   
    model_dir = '/oasis/projects/nsf/csd395/yuncong/jiaxu_repo/model_publish'
    init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.cpu())
else:
    model_dir = '/home/yuncong/jiaxu_repo/model_publish' 
    init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.gpu())

In [11]:
import time

In [ ]:
logging.basicConfig(level=logging.DEBUG)

# for sec in range(154, 169):
for sec in range(155, 169):

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    # if the dataset has more than one batch's amount of data, then you can see effect of multithreading enabled by openblas

    batch_size = 16

    t = time.time()

    test_iter = mx.io.ImageRecordIter(
        path_imgrec='/home/yuncong/CSHL_data_patches/%(dataset)s_test.rec' % {'dataset': dataset},
        batch_size=batch_size,
        data_shape=(3, 224, 224),
        mean_img=os.path.join(model_dir, 'mean_224.nd'),
    )

    print time.time() - t

    t = time.time()

    probs = init_model.predict(test_iter)
    
    # on Gordon, predicting 1 patch takes .5 s; OPENBLAS_NUM_THREADS=16 but CPUs are not fully utilized
    # on mbp, predicting 16 patches takes 1 s

    print time.time() - t
    
    # prediction_map = zip(np.argmax(probs, axis=1), map(lambda x: int(x[:-4]), img_filenames))
    np.save('/home/yuncong/CSHL_data_patches/%(dataset)s_test_prediction_map.npy'% {'dataset': dataset}, probs)

    # 900 sec = 15 min for 20k patches